# Packages 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'

sys.path.append(base_dir)
import os
from utils import *

import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import pickle

from annoy import AnnoyIndex
import polars as pl
import implicit
import scipy.sparse as sps
from src.eval import get_recall_at_k, pd_get_recall_at_k, model_eval
from src.config import raw_data_session_id_dir, candidate_dir, model_for_eval

/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Config 

In [2]:
debug = False

model_version = 'als_v1'

topn = 100
if debug:
    n_rows = 1000
else:
    n_rows = None
train_data_dir = '.'
test_data_dir = '.'
task = 'task1'

model_for_eval = True

model_dir = f'../model_training/{model_version}'

model_file = os.path.join(model_dir, 'als_model.pkl')

target_locals = ["DE", 'JP', 'UK']

submit_file = os.path.join('../data/sub_files/', 'als_v1.parquet')


In [3]:
! mkdir {model_dir}

mkdir: cannot create directory ‘../model_training/als_v1’: File exists


In [4]:
# model_file

# Read data

In [5]:
# train_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# # df_sess.head(3).collect()
# test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))


In [6]:
# train_data = train_pl.with_columns(
#     pl.col('prev_items').arr.concat(pl.col('next_item')) 
# )
# if not model_for_eval:
#     eval_data = eval_pl.with_columns(
#     pl.col('prev_items').arr.concat(pl.col('next_item')) 
# )
# else:
#     eval_data = eval_pl
# test_data = test_pl
# cols_to_keep = ['prev_items', 'locale']
# popular_pl = pl.concat([train_data.select(cols_to_keep), eval_data.select(cols_to_keep), test_data.select(cols_to_keep)], how='vertical')
        

In [7]:
# debug_session_num = 1000
# train_sessions = read_train_data(train_data_dir=train_data_dir)
# test_sessions = read_test_data(task, test_data_dir=test_data_dir)
# train_sessions = train_sessions[train_sessions['locale'].isin(target_locals)]

# if debug:
#     train_sessions = train_sessions.sample(debug_session_num)
#     test_sessions = test_sessions.sample(debug_session_num)
# f"{train_sessions.shape}; {test_sessions.shape}"

In [8]:
# with_columns(pl.col('prev_items').apply(str2list))

In [9]:
# str2list

In [10]:
train_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list)).collect()

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list)).collect()

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list)).collect()
product_pl = pl.scan_parquet(
    os.path.join(base_dir, raw_data_session_id_dir, 'products_train.parquet'), 
)
with open(os.path.join(base_dir, raw_data_session_id_dir, 'product_unique2id.json'), 'rb') as f:
    product_unique2id  = pickle.load(f)

In [11]:
train_sessions = train_pl.with_columns(
    pl.col('prev_items').arr.concat(pl.col('next_item')) 
).to_pandas()
if not model_for_eval:
    eval_sessions = eval_pl.with_columns(
    pl.col('prev_items').arr.concat(pl.col('next_item')) 
).to_pandas()
else:
    eval_sessions = eval_pl.to_pandas()
test_sessions = test_pl.to_pandas()

In [12]:
train_sessions['type'] = 'train'
eval_sessions['type'] = 'eval'
test_sessions['type'] = 'test'
train_sessions.reset_index(inplace=True)
eval_sessions.reset_index(inplace=True)
test_sessions.reset_index(inplace=True)

In [13]:
# train_sessions.head()

In [14]:
# test_sessions.head()

In [15]:
# train_sessions[cols_to_keep].head()

In [16]:
# train_df.head()

In [17]:
cols_to_keep = ['index', 'type', 'prev_items', 'locale']

In [18]:
original_train_test_df = pd.concat([train_sessions[cols_to_keep], eval_sessions[cols_to_keep], 
                                    test_sessions[cols_to_keep]], 
                          axis=0)

In [19]:
original_train_test_df.head()

,index,type,prev_items,locale
0,0,train,"[B09W9FND7K, B09JSPLN1M, B09M7GY217]",DE
1,1,train,"[B076THCGSG, B007MO8IME, B08MF65MLV, B001B4TKA...",DE
2,2,train,"[B0B1LGXWDS, B00AZYORS2, B0B1LGXWDS, B00AZYORS...",DE
3,3,train,"[B0749V8TC7, B0749W93VC, B0749TX4YP, B0749TX4YS]",DE
4,4,train,"[B09SMK3R8H, B01N4ND0F9, B08YNZT93Z]",DE


In [20]:
train_test_pl = pl.from_pandas(original_train_test_df)

In [21]:
def process_prev_item(prev_items):
    # print(prev_items)
    res = [ele.replace('[', '').replace(']', '').replace('\n', '').replace("'", '').replace(' ', '') for ele in prev_items.split(' ')]
    return res

In [22]:
train_test_pl = (
    train_test_pl
        # .with_columns(pl.col('prev_items').apply(lambda row: process_prev_item(row)))
        .with_columns(pl.col('index').cast(pl.Utf8))
        .with_columns((pl.col('index')+pl.col('type')).alias('user'))
        # .with_columns(pl.col('prev_items').explode())
)

In [23]:
train_test_df = train_test_pl.to_pandas()

In [24]:
train_test_df.head()

,index,type,prev_items,locale,user
0,0,train,"[B09W9FND7K, B09JSPLN1M, B09M7GY217]",DE,0train
1,1,train,"[B076THCGSG, B007MO8IME, B08MF65MLV, B001B4TKA...",DE,1train
2,2,train,"[B0B1LGXWDS, B00AZYORS2, B0B1LGXWDS, B00AZYORS...",DE,2train
3,3,train,"[B0749V8TC7, B0749W93VC, B0749TX4YP, B0749TX4YS]",DE,3train
4,4,train,"[B09SMK3R8H, B01N4ND0F9, B08YNZT93Z]",DE,4train


In [25]:
train_test_df.head()

,index,type,prev_items,locale,user
0,0,train,"[B09W9FND7K, B09JSPLN1M, B09M7GY217]",DE,0train
1,1,train,"[B076THCGSG, B007MO8IME, B08MF65MLV, B001B4TKA...",DE,1train
2,2,train,"[B0B1LGXWDS, B00AZYORS2, B0B1LGXWDS, B00AZYORS...",DE,2train
3,3,train,"[B0749V8TC7, B0749W93VC, B0749TX4YP, B0749TX4YS]",DE,3train
4,4,train,"[B09SMK3R8H, B01N4ND0F9, B08YNZT93Z]",DE,4train


In [26]:
train_test_df['user'] = train_test_df['user'].astype("category")
train_test_df['user_id'] = train_test_df['user'].cat.codes

In [27]:
test_sessions['index'] = test_sessions['index'].astype(str)
test_sessions = test_sessions.merge(train_test_df[['index', 'type', 'user_id']], how='left', on=['index', 'type'])

In [28]:
train_sessions['index'] = train_sessions['index'].astype(str)
train_sessions = train_sessions.merge(train_test_df[['index', 'type', 'user_id']], how='left', on=['index', 'type'])

In [29]:
eval_sessions['index'] = eval_sessions['index'].astype(str)
eval_sessions = eval_sessions.merge(train_test_df[['index', 'type', 'user_id']], how='left', on=['index', 'type'])

In [30]:
train_test_df = train_test_df.explode('prev_items')
train_test_df.shape

(18395684, 6)

In [31]:
test_sessions.head()

,index,prev_items,locale,session_id,type,user_id
0,0,"[B08V12CT4C, B08V1KXBQD, B01BVG1XJS, B09VC5PKN...",DE,3606249,test,1
1,1,"[B00R9R5ND6, B00R9RZ9ZS, B00R9RZ9ZS]",DE,3606250,test,1333334
2,2,"[B07YSRXJD3, B07G7Q5N6G, B08C9Q7QVK, B07G7Q5N6G]",DE,3606251,test,2612940
3,3,"[B08KQBYV43, 3955350843, 3955350843, 395535086...",DE,3606252,test,2789687
4,4,"[B09FPTCWMC, B09FPTQP68, B08HMRY8NG, B08TBBQ4B...",DE,3606253,test,2923020


In [32]:
train_test_df.head()

,index,type,prev_items,locale,user,user_id
0,0,train,B09W9FND7K,DE,0train,2
0,0,train,B09JSPLN1M,DE,0train,2
0,0,train,B09M7GY217,DE,0train,2
1,1,train,B076THCGSG,DE,1train,1333335
1,1,train,B007MO8IME,DE,1train,1333335


In [33]:


train_test_df['prev_items'] = train_test_df['prev_items'].astype("category")
train_test_df['item_id'] = train_test_df['prev_items'].cat.codes

In [34]:
train_test_df.head()

,index,type,prev_items,locale,user,user_id,item_id
0,0,train,B09W9FND7K,DE,0train,2,1096743
0,0,train,B09JSPLN1M,DE,0train,2,965099
0,0,train,B09M7GY217,DE,0train,2,993245
1,1,train,B076THCGSG,DE,1train,1333335,328622
1,1,train,B007MO8IME,DE,1train,1333335,110154


In [35]:
len(train_test_df)

18395684

In [36]:
itemid2item = dict(zip(train_test_df['item_id'], train_test_df['prev_items']))

In [37]:
len(itemid2item)

1335037

In [38]:
max(itemid2item.keys())

1335036

In [39]:
# itemid2item = 
# train_test_df.groupby(['prev_items', 'item_id'])['user'].count().reset_index()

In [40]:
# itemid2item

# ALS Model

In [41]:
train_test_df.head()

,index,type,prev_items,locale,user,user_id,item_id
0,0,train,B09W9FND7K,DE,0train,2,1096743
0,0,train,B09JSPLN1M,DE,0train,2,965099
0,0,train,B09M7GY217,DE,0train,2,993245
1,1,train,B076THCGSG,DE,1train,1333335,328622
1,1,train,B007MO8IME,DE,1train,1333335,110154


In [42]:
# user_item[1243, :]

In [43]:
user_item = sps.coo_matrix(
      (
          np.ones(train_test_df.shape[0]), # We're using a matrix of ones, but using type weights or repurchase weights could help!
          (train_test_df['user_id'],
          train_test_df['item_id'])
      ),
      dtype='int8'
    ).tocsr()


In [44]:
user_item.shape


(3589687, 1335037)

In [45]:
model = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.1, iterations=20)

model.fit(user_items=user_item, show_progress=True)

  0%|          | 0/20 [00:00<?, ?it/s]

In [46]:
# help(model.recommend_all)

In [47]:
# help(model.recommend)

In [48]:
def get_rec(user_id, model, user_item, itemid2item, topn=100):
    item_ids = model.recommend(user_id, 
                              user_item[user_id, :],
                              filter_already_liked_items=True,
                               N=topn
                             )[0].tolist()
    # print(item_ids)
    items = [itemid2item[key] for key in item_ids]
    return items
    

In [49]:
assert model.item_factors.shape[0] == len(itemid2item)

In [50]:
# itemid2item[7550]

In [51]:
# itemid2item


In [52]:
# help(model.recommend)

In [53]:
# original_train_test_df

In [54]:
# test_prediction = train_test_df[(train_test_df['type']=='test')]['item_id'].unique()

In [55]:
# test_sessions.shape

In [56]:
# predictions.head()

## Save model 

In [57]:
debug

False

In [58]:
model_dir

'../model_training/als_v1'

In [59]:
# sps.save_npz(os.path.join(model_dir, "user_item.npz"), user_item)
# with open(model_file, 'wb') as f:
#     pickle.dump(model, f)
# with open(os.path.join(model_dir, 'itemid2item.json'), 'wb') as f:
#     pickle.dump(itemid2item, f)
# del model
# del user_item
# del itemid2item

# user_item = sps.load_npz(os.path.join(model_dir, "user_item.npz"))
# with open(model_file, 'rb') as f:
#     model = pickle.load(f)
# with open(os.path.join(model_dir, 'itemid2item.json'), 'rb') as f:
#     itemid2item = pickle.load(f)

In [60]:
if not debug:
    sps.save_npz(os.path.join(model_dir, "user_item.npz"), user_item)
    with open(model_file, 'wb') as f:
        pickle.dump(model, f)
    with open(os.path.join(model_dir, 'itemid2item.json'), 'wb') as f:
        pickle.dump(itemid2item, f)
    del model
    del user_item
    del itemid2item
    
    user_item = sps.load_npz(os.path.join(model_dir, "user_item.npz"))
    with open(model_file, 'rb') as f:
        model = pickle.load(f)
    with open(os.path.join(model_dir, 'itemid2item.json'), 'rb') as f:
        itemid2item = pickle.load(f)

In [61]:
model_file

'../model_training/als_v1/als_model.pkl'

In [62]:
model

## Model eval 

In [63]:
# %%time
# train_df = (
#     pl.from_pandas(train_sessions.sample(300))
#         .with_columns(pl.col('user_id').apply(lambda row: get_rec(row, model=model, user_item=user_item
#                                                                  , itemid2item=itemid2item)).alias('next_item_prediction'))
# ).to_pandas()#[['locale', 'next_item_prediction']]

In [64]:

# eval_cols = ['len', 'recall@20', 'recall@100']
# train_df[eval_cols] = train_df.apply(pd_get_recall_at_k, axis=1, result_type='expand')
# print(train_df[eval_cols].mean())

# Model Eval

In [65]:
# train_sessions.head()

In [66]:
# test_sessions.head()

In [67]:
# train_sessions.head()

In [68]:
eval_sessions.head()

,index,prev_items,next_item,locale,session_id,type,user_id
0,0,"[B09XMTWDVT, B0B4MZZ8MB, B0B7HZ2GWX, B09XMTWDV...",B0B4R9NN4B,DE,3,eval,0
1,1,"[B09Y5CSL3T, B09Y5DPTXN, B09FKD61R8]",B0BGVBKWGZ,DE,4,eval,1333333
2,2,"[B07LFQPX5L, B07LFPJTVF]",B07LFRP5SS,DE,14,eval,2612939
3,3,"[B085QWM3KB, B001BAAV5W, B002GHKPQY, B010VC4AG...",B000VVSWBM,DE,15,eval,2789686
4,4,"[B01NAQR0Y2, B08XBZL5CM]",B09ZPNHQ12,DE,27,eval,2923019


In [69]:
# test_sessions.head()

In [70]:
%%time
eval_res = (
    pl.from_pandas(eval_sessions)
        .with_columns(pl.col('user_id').apply(lambda row: get_rec(row, model=model, user_item=user_item
                                                                 , itemid2item=itemid2item)).alias('next_item_prediction'))
).to_pandas()

CPU times: user 22h 56min 41s, sys: 9h 49min 59s, total: 1d 8h 46min 41s
Wall time: 1h 11min 27s


In [71]:
# eval_res.head()

In [72]:
model_eval(pl.from_pandas(eval_res))

total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
326443,0.004352,0.013034,0.029938


In [73]:
eval_res.to_parquet('../data/als_v1_eval.parquet', engine='pyarrow')

In [74]:
! ls ../data/

als_v1_eval.parquet  eval_data	      raw_data		   sub_files
candidates	     rank_train_data  raw_data_session_id  train_data


In [ ]:
%%time
predictions = (
    pl.from_pandas(test_sessions)
        .with_columns(pl.col('user_id').apply(lambda row: get_rec(row, model=model, user_item=user_item
                                                                 , itemid2item=itemid2item)).alias('next_item_prediction'))
).to_pandas()[['locale', 'next_item_prediction']]

# Rec & Save Result 

In [ ]:
submit_file

In [ ]:
check_predictions(predictions, test_sessions=test_sessions, 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend
predictions.to_parquet(submit_file, engine='pyarrow')

# Submission 

In [ ]:
# You can submit with aicrowd-cli, or upload manually on the challenge page.
!aicrowd submission create -c task-1-next-product-recommendation -f {submit_file}

# Example 

In [ ]:
# # -*- coding: utf-8 -*-
# """
# Created on Sun Jun 23 22:20:58 2019

# @author: himansh
# """
# #import libraries
# import sys
# import pandas as pd
# import numpy as np
# import scipy.sparse as sparse
# from scipy.sparse.linalg import spsolve
# import random



# from sklearn.preprocessing import MinMaxScaler

# import implicit 
# from datetime import datetime, timedelta


# #Data Preprocessing
# def create_data(datapath,start_date,end_date):
#     df=pd.read_csv(datapath)
#     df=df.assign(date=pd.Series(datetime.fromtimestamp(a/1000).date() for a in df.timestamp))
#     df=df.sort_values(by='date').reset_index(drop=True) # for some reasons RetailRocket did NOT sort data by date
#     df=df[(df.date>=datetime.strptime(start_date,'%Y-%m-%d').date())&(df.date<=datetime.strptime(end_date,'%Y-%m-%d').date())]
#     df=df[['visitorid','itemid','event']]
#     return df


In [ ]:
# #Download the kaggle RetailRocket data and give the events.csv file path
# datapath= 'events.csv'
# data=create_data(datapath,'2015-5-3','2015-5-3')
# data['visitorid'] = data['visitorid'].astype("category")
# data['itemid'] = data['itemid'].astype("category")
# data['visitor_id'] = data['visitorid'].cat.codes
# data['item_id'] = data['itemid'].cat.codes

# data['event']=data['event'].astype('category')
# data['event']=data['event'].cat.codes


In [ ]:
# sparse_item_user = sparse.csr_matrix((data['event'].astype(float), (data['item_id'], data['visitor_id'])))
# sparse_user_item = sparse.csr_matrix((data['event'].astype(float), (data['visitor_id'], data['item_id'])))


In [ ]:



# #Building the model
# model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)


# # ???
# # alpha_val = 40
# # data_conf = (sparse_item_user * alpha_val).astype('double')


In [ ]:
# ###USING THE MODEL

# #Get Recommendations
# user_id =   14
# recommended = model.recommend(user_id, 
#                               sparse_user_item[user_id],
#                               filter_already_liked_items=True
#                              )[0].tolist()
# print(recommended)




# # #Get similar items
# # item_id = 7
# # n_similar = 3
# # similar = model.similar_items(item_id, n_similar)
# # print(similar)

In [ ]:
# user_item = sps.coo_matrix(
#       (
#           np.ones(data.shape[0]), # We're using a matrix of ones, but using type weights or repurchase weights could help!
#           (data['visitor_id'],
#           data['item_id'])
#       ),
#       dtype='int8'
#     ).tocsr()
# user_item.todense().shape
# new_model = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.1, iterations=20)

# new_model.fit(user_items=user_item, show_progress=True)


In [ ]:
# user_id =   14
# recommended = model.recommend(user_id, 
#                               user_item[user_id, :],
#                               filter_already_liked_items=True
#                              )[0].tolist()
# print(recommended)